In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

import chess
import numpy as np
import board_representation.sentimate as br
import torch
from data_modules.move_as_tensor import MoveAsTensorDatasetLazy
from data_modules.lightning import MoveEvaluationDataModule

from utils.utils import load_sql_to_df

import torch
import models.move_evaluation as me_models
import models.lightning as pl_models
import pytorch_lightning as pl

from datasets import Dataset
from torch.utils.data import DataLoader
from data_modules.lightning import MoveEvaluationDataModule
from sklearn.model_selection import train_test_split

from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning.callbacks import RichProgressBar

In [3]:
db_filename = "./../../data/databases/chess_moves_quality.db"
moves_df = load_sql_to_df("SELECT * FROM sentimate_v1", db_filename)
moves_df = moves_df.rename(columns={"fen": "position"})

In [16]:
load_sql_to_df("SELECT * FROM comments_and_nags", db_filename)

,fen,move,sentiment
0,b2r3r/4Rp1p/p4np1/1p1q4/kP6/P1Q2PPB/2P4P/1K6 w...,e7a7,1
1,7r/3R1p1p/6p1/1p6/2q5/5PP1/1Q1r3P/1K1k1B2 b - ...,d2d7,1
2,r3r3/3n1p1p/pR4p1/1p1q4/kP6/P1Q2PP1/2P4P/1K3B2...,b6d6,1
3,3r3r/1R3p1p/p5p1/1p6/kPq1P3/P1Q3PB/2P4P/1K6 w ...,c3c4,0
4,b2r3r/R4p1p/p2q2p1/1p1n4/kP1Q4/P4PPB/2P4P/1K6 ...,a7a6,1
...,...,...,...
6488302,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,c2c4,1
6488303,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,c2c4,1
6488304,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,c2c4,1
6488305,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,c2c4,1


In [4]:
from multiprocessing import Pool

def test_move(move):
    # print(move)
    try:
        board = chess.Board(move["position"])
        board.push(chess.Move.from_uci(move["move"]))
        return True
    except:
        return False
    
def apply_filter_multiprocessing(df):
    with Pool(15) as pool:
        result = pool.map(test_move, [row for index, row in df.iterrows()])
    return df[result]


In [5]:
moves_df

,position,move,sentiment
0,b2r3r/4Rp1p/p4np1/1p1q4/kP6/P1Q2PPB/2P4P/1K6 w...,e7a7,1
1,7r/3R1p1p/6p1/1p6/2q5/5PP1/1Q1r3P/1K1k1B2 b - ...,d2d7,1
2,r3r3/3n1p1p/pR4p1/1p1q4/kP6/P1Q2PP1/2P4P/1K3B2...,b6d6,1
3,3r3r/1R3p1p/p5p1/1p6/kPq1P3/P1Q3PB/2P4P/1K6 w ...,c3c4,0
4,b2r3r/R4p1p/p2q2p1/1p1n4/kP1Q4/P4PPB/2P4P/1K6 ...,a7a6,1
...,...,...,...
11089181,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,b2b3,0
11089182,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,d4c6,0
11089183,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,h2h4,0
11089184,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,d4b5,0


In [5]:
moves_df_2 = apply_filter_multiprocessing(moves_df)
moves_df_2

,position,move,sentiment
0,b2r3r/4Rp1p/p4np1/1p1q4/kP6/P1Q2PPB/2P4P/1K6 w...,e7a7,1
1,7r/3R1p1p/6p1/1p6/2q5/5PP1/1Q1r3P/1K1k1B2 b - ...,d2d7,1
2,r3r3/3n1p1p/pR4p1/1p1q4/kP6/P1Q2PP1/2P4P/1K3B2...,b6d6,1
3,3r3r/1R3p1p/p5p1/1p6/kPq1P3/P1Q3PB/2P4P/1K6 w ...,c3c4,0
4,b2r3r/R4p1p/p2q2p1/1p1n4/kP1Q4/P4PPB/2P4P/1K6 ...,a7a6,1
...,...,...,...
11089181,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,b2b3,0
11089182,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,d4c6,0
11089183,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,h2h4,0
11089184,r1bqkbnr/pp1ppp1p/2n3p1/8/3NP3/8/PPP2PPP/RNBQK...,d4b5,0


In [6]:
data_module = MoveEvaluationDataModule(dataframe=moves_df, batch_size=512, num_workers=15)

In [ ]:
data_module.setup()

In [ ]:
data_module.batch_size = 1024

In [24]:
model = me_models.SentimateNet(output_size=2, dropout=0.1, skip_connection=True, input_channels=24)
# model = me_models.AlphaZeroOnlyValueHead(num_blocks=2, output_size=2)
pl_model = pl_models.PLClassifierModel(model, learning_rate=0.001, save_dir=None)

In [25]:
tensorboard_logger = TensorBoardLogger(save_dir="../../lightning_logs/sentimate/skip_connection", name=f"{type(pl_model.model).__name__}")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=10,
    callbacks=[RichProgressBar()],
    logger=tensorboard_logger,
    # default_root_dir=
    # fast_dev_run=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    model=pl_model,
    datamodule=data_module,
    # ckpt_path="../../lightning_logs/sentimate/SentimateNet/version_0/checkpoints/epoch=49-step=536050.ckpt"
    # ckpt_path="../../lightning_logs/sentimate/tensorboard/SentimateNet/version_1/checkpoints/epoch=29-step=321630.ckpt"
)